In [22]:
library(tidyverse)

# plots

In [23]:
pca_plot <- function(df, batch_info, title, path) {
  pca <- prcomp(t(na.omit(df)))
  # Plot PCA
  pca_df <-
    pca$x %>%
    as.data.frame() %>%
    rownames_to_column("file") %>% 
    left_join(batch_info,  by = "file") 
  # add % of explained variance
  var_expl <- pca$sdev^2 / sum(pca$sdev^2)
  names(var_expl) <- paste0("PC", 1:length(var_expl))
  # Add the label for the specific point
  pca_plot <- pca_df %>%
    ggplot(aes(PC1, PC2)) +
    geom_point(aes(col=condition, shape=lab), size=2) +
    theme_classic() +
    labs(title = title,
         x = glue::glue("PC1 [{round(var_expl['PC1']*100, 2)}%]"),
         y = glue::glue("PC2 [{round(var_expl['PC2']*100, 2)}%]"))
  
   # Check if "S37" exists in the pca_df dataframe
  if("Ref8537_S37" %in% pca_df$file) {
    pca_plot <- pca_plot +
      geom_text(data = pca_df[pca_df$file == "Ref8537_S37", ], aes(label = 'S37'), vjust = -1)
  }

  ggsave(path, pca_plot)
}

In [24]:
# boxplot
boxplot_pg <- function(protein_matrix, title, path) {
  # Reshape data into long format
  long_data <- tidyr::gather(protein_matrix, 
                             key = "file", value = "Intensity")
  # Log tranformed scale
  boxplot <- ggplot(long_data, aes(x = file, y = Intensity)) + 
    geom_boxplot() +
    stat_summary(fun = mean, geom = "point", shape = 4, size = 1.5, color = "red") +
    theme_bw() +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1, size = 5)) +
    labs(title = title) 

  ggsave(path, boxplot, width = 6, height = 6)
}

In [25]:
heatmap_plot <- function(pg_matrix, batch_info, name, plot_name_prefix){
    cor_matrix <- cor(na.omit(pg_matrix), use = "pairwise.complete.obs")
    pheatmap::pheatmap(cor_matrix, 
                        annotation_col = select(batch_info, c(condition, lab)),
                        treeheight_row = 0, treeheight_col = 0, 
                        fontsize_row = 5, fontsize_col = 5,
                        width = 7, height = 7,
                        main = paste0(name, ' heatmap'),
                        filename = plot_name_prefix)
}


In [26]:
plot_three_plots <- function(pg_matrix, batch_info, plot_name_prefix, subname, number){

        batch_info <- batch_info %>%
                mutate(file = case_when(
                lab %in% c('lab_A', 'lab_E') ~ str_split(file, "_") %>% 
                map_chr(~ if (length(.x) == 4) paste(.x[1], .x[2], .x[4], sep = "_") else paste(.x[1], .x[2], sep = "_")),
                
                lab == 'lab_C' ~ str_split(file, "_") %>% 
                map_chr(~ paste(.x[5], .x[6], sep = "_")),
                
                lab == 'lab_D' ~ str_split(file, "_") %>% 
                map_chr(~ paste(.x[6], .x[8], sep = "_")),
                
                TRUE ~ file
        ))

        rownames(batch_info) <- batch_info$file
        colnames(pg_matrix) <- batch_info$file


        boxplot_pg(pg_matrix, 
                paste0(subname, ' boxplot'), 
                paste0(plot_name_prefix, "_", subname, "_", number, "_boxplot.png")
        )

        pca_plot(pg_matrix,
                batch_info, 
                paste0(subname, ' pca'), 
                paste0(plot_name_prefix, "_", subname, "_", number, "_pca.png")
        )

        heatmap_plot(pg_matrix, 
                batch_info, 
                subname, 
                paste0(plot_name_prefix, "_", subname, "_", number, "_heatmap.png")
        )

}

# data

In [27]:
pg_matrix <- read.csv("/home/yuliya/repos/cosybio/removeBatch/test_data/all_pg_matrix.csv", row.names = 1)
dim(pg_matrix)
head(pg_matrix)

[1] 3110   99

,Ref8537_QC1_20230414_1,Ref8537_QC1_20230414_2,Ref8537_QC2_20230414_1,Ref8537_QC2_20230414_2,Ref8537_QC3_20230414_1,Ref8537_QC3_20230414_2,Ref8537_QC4_20230414_1,Ref8537_QC4_20230414_2,Ref8537_S4_20230414,Ref8537_S8_20230414,⋯,CVT09_s55_X012,CVT09_s59_X013,CVT09_s61_X014,CVT09_s67_X015,CVT09_s73_X016,CVT09_s77_X017,CVT09_s84_X018,CVT09_s90_X019,CVT09_s93_X020,CVT09_s99_X021
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
P0A8D6,23.93735,23.84543,23.87828,24.02926,23.75127,23.60130,23.82125,23.73943,23.78537,23.82508,⋯,21.38164,21.39400,21.47776,21.27816,21.37711,21.38204,21.52550,21.32760,21.34347,21.49474
P0AAX3,22.34260,22.39116,22.35547,22.16531,23.27452,23.34318,22.87165,22.88306,22.43577,21.98993,⋯,20.48433,20.88107,20.55610,20.50723,20.35750,20.56573,20.75241,20.86534,20.23317,20.49141
P00959,26.86459,26.89579,26.93303,26.89099,27.12835,27.11182,27.20538,27.18897,26.86022,26.91073,⋯,23.35523,23.35402,23.34067,23.43078,23.40860,23.39242,23.37831,23.42953,23.36019,23.37459
P0AEQ1,23.26479,23.40751,23.15299,23.06521,21.88854,22.07164,21.95835,22.00192,23.55970,23.42499,⋯,18.90326,18.93703,19.12520,18.79174,18.91994,19.00150,18.84693,18.86085,18.77401,18.94979
P25738,25.05893,25.27839,24.46547,24.55379,24.42509,24.43553,24.19156,24.36375,26.01347,25.76989,⋯,22.70138,22.32526,22.86180,22.50757,22.59620,22.59773,22.31102,22.44497,22.49490,22.47742
P64596,26.01417,26.05369,25.77075,25.72445,25.40850,25.34799,25.51174,25.44203,26.13285,26.08724,⋯,22.29860,22.31546,22.22938,22.35725,22.29671,22.38484,22.27541,22.30215,22.35032,22.30993


In [28]:
# pg_matrix <- na.omit(pg_matrix)
# dim(pg_matrix)
# head(pg_matrix)

In [29]:
batch_info_ref <- read.csv("/home/yuliya/repos/cosybio/removeBatch/test_data/batch_info_all_ref.csv") %>%
    mutate(lab = factor(lab), condition = factor(condition))
rownames(batch_info_ref) <- batch_info_ref$file
dim(batch_info_ref)
head(batch_info_ref)

[1] 99  3

,file,condition,lab
,<chr>,<fct>,<fct>
Ref8537_QC1_20230414_1,Ref8537_QC1_20230414_1,A_QC,lab_A
Ref8537_QC1_20230414_2,Ref8537_QC1_20230414_2,A_QC,lab_A
Ref8537_QC2_20230414_1,Ref8537_QC2_20230414_1,A_QC,lab_A
Ref8537_QC2_20230414_2,Ref8537_QC2_20230414_2,A_QC,lab_A
Ref8537_QC3_20230414_1,Ref8537_QC3_20230414_1,B_QC,lab_A
Ref8537_QC3_20230414_2,Ref8537_QC3_20230414_2,B_QC,lab_A


In [30]:
batch_info <- read.csv("/home/yuliya/repos/cosybio/removeBatch/test_data/batch_info_all.csv") %>%
    mutate(lab = factor(lab), condition = factor(condition))
rownames(batch_info)<- batch_info$file
dim(batch_info)
head(batch_info)

[1] 99  3

,file,condition,lab
,<chr>,<fct>,<fct>
Ref8537_QC1_20230414_1,Ref8537_QC1_20230414_1,A,lab_A
Ref8537_QC1_20230414_2,Ref8537_QC1_20230414_2,A,lab_A
Ref8537_QC2_20230414_1,Ref8537_QC2_20230414_1,A,lab_A
Ref8537_QC2_20230414_2,Ref8537_QC2_20230414_2,A,lab_A
Ref8537_QC3_20230414_1,Ref8537_QC3_20230414_1,B,lab_A
Ref8537_QC3_20230414_2,Ref8537_QC3_20230414_2,B,lab_A


In [31]:
plot_name_prefix <- paste0("/home/yuliya/repos/cosybio/removeBatch/plots/", "R_before_correction")
subname <- "A_B_conditions"
number <- "01"
plot_three_plots(pg_matrix, batch_info_ref, plot_name_prefix, subname, number)


Warning message:
“Removed 53437 rows containing non-finite values (`stat_boxplot()`).”
Warning message:
“Removed 53437 rows containing non-finite values (`stat_summary()`).”
Saving 6.67 x 6.67 in image


# correction

In [32]:
library(limma)

In [33]:
pg_matrix <- read.csv("/home/yuliya/repos/cosybio/removeBatch/test_data/all_pg_matrix.csv", row.names = 1)
dim(pg_matrix)
head(pg_matrix)

[1] 3110   99

,Ref8537_QC1_20230414_1,Ref8537_QC1_20230414_2,Ref8537_QC2_20230414_1,Ref8537_QC2_20230414_2,Ref8537_QC3_20230414_1,Ref8537_QC3_20230414_2,Ref8537_QC4_20230414_1,Ref8537_QC4_20230414_2,Ref8537_S4_20230414,Ref8537_S8_20230414,⋯,CVT09_s55_X012,CVT09_s59_X013,CVT09_s61_X014,CVT09_s67_X015,CVT09_s73_X016,CVT09_s77_X017,CVT09_s84_X018,CVT09_s90_X019,CVT09_s93_X020,CVT09_s99_X021
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
P0A8D6,23.93735,23.84543,23.87828,24.02926,23.75127,23.60130,23.82125,23.73943,23.78537,23.82508,⋯,21.38164,21.39400,21.47776,21.27816,21.37711,21.38204,21.52550,21.32760,21.34347,21.49474
P0AAX3,22.34260,22.39116,22.35547,22.16531,23.27452,23.34318,22.87165,22.88306,22.43577,21.98993,⋯,20.48433,20.88107,20.55610,20.50723,20.35750,20.56573,20.75241,20.86534,20.23317,20.49141
P00959,26.86459,26.89579,26.93303,26.89099,27.12835,27.11182,27.20538,27.18897,26.86022,26.91073,⋯,23.35523,23.35402,23.34067,23.43078,23.40860,23.39242,23.37831,23.42953,23.36019,23.37459
P0AEQ1,23.26479,23.40751,23.15299,23.06521,21.88854,22.07164,21.95835,22.00192,23.55970,23.42499,⋯,18.90326,18.93703,19.12520,18.79174,18.91994,19.00150,18.84693,18.86085,18.77401,18.94979
P25738,25.05893,25.27839,24.46547,24.55379,24.42509,24.43553,24.19156,24.36375,26.01347,25.76989,⋯,22.70138,22.32526,22.86180,22.50757,22.59620,22.59773,22.31102,22.44497,22.49490,22.47742
P64596,26.01417,26.05369,25.77075,25.72445,25.40850,25.34799,25.51174,25.44203,26.13285,26.08724,⋯,22.29860,22.31546,22.22938,22.35725,22.29671,22.38484,22.27541,22.30215,22.35032,22.30993


In [34]:
a_prots <- rownames(pg_matrix[,batch_info[batch_info$lab == 'lab_A',]$file] %>% filter(rowSums(is.na(.)) != ncol(.)))
c_prots <- rownames(pg_matrix[,batch_info[batch_info$lab == 'lab_C',]$file] %>% filter(rowSums(is.na(.)) != ncol(.)))
d_prots <- rownames(pg_matrix[,paste0('X', batch_info[batch_info$lab == 'lab_D',]$file)] %>% filter(rowSums(is.na(.)) != ncol(.)))
e_prots <- rownames(pg_matrix[,batch_info[batch_info$lab == 'lab_E',]$file] %>% filter(rowSums(is.na(.)) != ncol(.)))

shared_prots <- intersect(a_prots, c_prots)
shared_prots <- intersect(shared_prots, d_prots)
shared_prots <- intersect(shared_prots, e_prots) %>% sort()

pg_matrix <- pg_matrix[shared_prots, ]

In [35]:
batch_info <- read.csv("/home/yuliya/repos/cosybio/removeBatch/test_data/batch_info_all.csv") %>%
    mutate(lab = factor(lab), condition = factor(condition))
rownames(batch_info)<- batch_info$file
dim(batch_info)
head(batch_info)

[1] 99  3

,file,condition,lab
,<chr>,<fct>,<fct>
Ref8537_QC1_20230414_1,Ref8537_QC1_20230414_1,A,lab_A
Ref8537_QC1_20230414_2,Ref8537_QC1_20230414_2,A,lab_A
Ref8537_QC2_20230414_1,Ref8537_QC2_20230414_1,A,lab_A
Ref8537_QC2_20230414_2,Ref8537_QC2_20230414_2,A,lab_A
Ref8537_QC3_20230414_1,Ref8537_QC3_20230414_1,B,lab_A
Ref8537_QC3_20230414_2,Ref8537_QC3_20230414_2,B,lab_A


In [36]:
batch_info$lab

[1] lab_A lab_A lab_A lab_A lab_A lab_A lab_A lab_A lab_A lab_A lab_A lab_A
[13] lab_A lab_A lab_A lab_A lab_A lab_A lab_A lab_A lab_A lab_A lab_A lab_A
[25] lab_A lab_A lab_A lab_A lab_C lab_C lab_C lab_C lab_C lab_C lab_C lab_C
[37] lab_C lab_C lab_C lab_C lab_C lab_C lab_C lab_C lab_C lab_C lab_C lab_C
[49] lab_C lab_C lab_C lab_D lab_D lab_D lab_D lab_D lab_D lab_D lab_D lab_D
[61] lab_D lab_D lab_D lab_D lab_D lab_D lab_D lab_D lab_D lab_D lab_D lab_D
[73] lab_D lab_D lab_D lab_E lab_E lab_E lab_E lab_E lab_E lab_E lab_E lab_E
[85] lab_E lab_E lab_E lab_E lab_E lab_E lab_E lab_E lab_E lab_E lab_E lab_E
[97] lab_E lab_E lab_E
Levels: lab_A lab_C lab_D lab_E

In [37]:
pg_matrix_cured <- removeBatchEffect(pg_matrix, batch_info[['lab']]) %>% as.data.frame()

In [38]:
dim(pg_matrix_cured)
head(pg_matrix_cured)

[1] 2336   99

,Ref8537_QC1_20230414_1,Ref8537_QC1_20230414_2,Ref8537_QC2_20230414_1,Ref8537_QC2_20230414_2,Ref8537_QC3_20230414_1,Ref8537_QC3_20230414_2,Ref8537_QC4_20230414_1,Ref8537_QC4_20230414_2,Ref8537_S4_20230414,Ref8537_S8_20230414,⋯,CVT09_s55_X012,CVT09_s59_X013,CVT09_s61_X014,CVT09_s67_X015,CVT09_s73_X016,CVT09_s77_X017,CVT09_s84_X018,CVT09_s90_X019,CVT09_s93_X020,CVT09_s99_X021
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A5A614,19.59722,19.63527,19.29085,19.22962,19.17200,18.82606,18.47446,18.69794,19.73327,19.80267,⋯,18.89630,NA,18.96889,18.88278,18.81896,18.44805,18.64991,NA,19.31842,18.45111
O32583,20.13940,20.32151,20.41263,20.32826,19.98819,19.88644,19.05726,20.19234,20.26307,19.80881,⋯,NA,NA,NA,19.15097,19.97514,NA,NA,NA,NA,NA
P00350,26.46399,26.54430,26.49451,26.53334,26.67013,26.66674,26.70791,26.73010,26.66500,26.58467,⋯,26.74266,26.79849,26.74247,26.75681,26.77118,26.76576,26.71698,26.74635,26.71519,26.74117
P00363,24.18471,24.19133,24.26651,24.25851,24.26212,24.31023,24.11584,24.11355,24.16982,24.11679,⋯,23.93092,23.81155,23.64283,23.86221,24.01495,23.83545,23.83399,23.92455,23.97780,23.80401
P00370,25.57339,25.58858,25.49763,25.47360,26.18039,26.13536,26.02937,25.97825,25.47854,25.43161,⋯,25.93167,25.92334,25.86242,25.78598,25.97907,25.95999,25.89339,25.93304,26.03555,25.77288
P00393,23.93275,23.94527,23.98156,23.97987,23.86229,23.86701,23.96535,24.00174,23.70935,23.94746,⋯,23.89696,23.78159,23.95530,23.78495,23.80077,23.90782,23.90680,23.85752,23.84160,23.97591


In [39]:
pg_matrix_cured %>% write.table("/home/yuliya/repos/cosybio/removeBatch/results/central_cured.csv", sep = "\t",)

In [40]:
plot_name_prefix <- paste0("/home/yuliya/repos/cosybio/removeBatch/plots/", "R_after_correction")
subname <- "A_B_conditions"
number <- "02"
plot_three_plots(pg_matrix_cured, batch_info_ref, plot_name_prefix, subname, number)


Warning message:
“Removed 4470 rows containing non-finite values (`stat_boxplot()`).”
Warning message:
“Removed 4470 rows containing non-finite values (`stat_summary()`).”
Saving 6.67 x 6.67 in image
